In [1]:
from sentence_transformers import SentenceTransformer

/workspaces/llm-zoomcamp/03-vector-search/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
# Q1
encoding = embedding_model.encode(user_question)
encoding[0]

0.078222655

In [5]:
# Q2 Load the Documents with IDs

import requests
from tqdm.auto import tqdm

docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for document in tqdm(documents_raw):
    if document['course'] == 'machine-learning-zoomcamp':
        documents.append(document)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [6]:
len(documents)

375

In [7]:
# Q2 create embeddings
embeddings = []

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

  0%|          | 0/375 [00:00<?, ?it/s]

In [8]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

In [9]:
# Q3 search
v = encoding
scores = X.dot(v)
scores.max()

0.6506573

In [10]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [13]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [14]:
# Q4 evaluate with Hit-rate
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [15]:
## calculate relevance
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v_doc_question = embedding_model.encode(q['question'])
    results = search_engine.search(v_doc_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [17]:
hit_rate(relevance_total)

0.9398907103825137

In [18]:
# Q5 indexing with elastic search
len(v)

768

In [28]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://127.0.0.1:9200', request_timeout=500) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "document": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [21]:
# add fields for embeddings to be saved
for doc in tqdm(ground_truth):
    question = doc['question']
    
    doc['question_vector'] = embedding_model.encode(question)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [22]:
es_client.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0})

In [46]:
es_client.indices.get_settings(index=index_name)

ObjectApiResponse({'course-questions': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'course-questions', 'creation_date': '1720989834391', 'number_of_replicas': '0', 'uuid': 'lrPJJIybRqKOS5eg83OOPw', 'version': {'created': '8040399'}}}}})

In [32]:
# index all documents with all fields
for doc in tqdm(ground_truth):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [33]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["question", "course", "document"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    print(es_results['hits']['hits'])
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [34]:
elastic_search_knn('question_vector', v)

[{'_index': 'course-questions', '_id': 'ecAgs5AB55xG7fn83DTx', '_score': 0.85088146, '_source': {'question': 'If I join the course late, can I still participate?', 'document': 'ee58a693', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'esAgs5AB55xG7fn83TQF', '_score': 0.8340343, '_source': {'question': 'Will I be able to obtain a certificate if I join the course after it has started?', 'document': 'ee58a693', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'M8Ags5AB55xG7fn81jSV', '_score': 0.81525576, '_source': {'question': 'Where can I sign up for the course?', 'document': '0227b872', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'f8Ags5AB55xG7fn83TRp', '_score': 0.805832, '_source': {'question': 'Can I start the course anytime?', 'document': '636f55d5', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'iMAgs5AB55xG7fn83jQf', '_score': 0.7835678, '_source': {'

[{'question': 'If I join the course late, can I still participate?',
  'document': 'ee58a693',
  'course': 'machine-learning-zoomcamp'},
 {'question': 'Will I be able to obtain a certificate if I join the course after it has started?',
  'document': 'ee58a693',
  'course': 'machine-learning-zoomcamp'},
 {'question': 'Where can I sign up for the course?',
  'document': '0227b872',
  'course': 'machine-learning-zoomcamp'},
 {'question': 'Can I start the course anytime?',
  'document': '636f55d5',
  'course': 'machine-learning-zoomcamp'},
 {'question': 'What is the initial step after joining the course?',
  'document': '0a278fb2',
  'course': 'machine-learning-zoomcamp'}]

In [41]:
# Q6 compare results of metrics with elastic search
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [42]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q)

evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

[{'_index': 'course-questions', '_id': 'M8Ags5AB55xG7fn81jSV', '_score': 1.0, '_source': {'question': 'Where can I sign up for the course?', 'document': '0227b872', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'W8Ags5AB55xG7fn82jQs', '_score': 0.8207463, '_source': {'question': 'How can I earn a certificate in this course?', 'document': '2eba08e3', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'NsAgs5AB55xG7fn81jTg', '_score': 0.81388736, '_source': {'question': 'Does this course have a GitHub repository for the sign-up link?', 'document': '0227b872', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'iMAgs5AB55xG7fn83jQf', '_score': 0.7935101, '_source': {'question': 'What is the initial step after joining the course?', 'document': '0a278fb2', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'icAgs5AB55xG7fn83jQ2', '_score': 0.78872633, '_source': {'question': 

{'hit_rate': 0.9972677595628415}